#


Gruppe 21, 

In [90]:
from secrets import token_bytes
import numpy as np
import pandas as pd
import nltk
import re
import nltk
from matplotlib import pyplot as plt 

In [91]:
df_load = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv", delimiter=",")

In [92]:
df = df_load.copy()
def clean_text(txt):
  txt = txt.lower()
  txt = re.sub(r"\s+", " ", txt)
  regex = {"DATE" : r"(\d+[\\/-])?\d+[\\/-]\d+",
      "EMAIL" : r"\w+@\w+\.[a-z]{2,}",
      "URL" : r"([a-z\d]+://)?[a-z]+[a-z\d]*(\.[a-z\d\-\._~]+)+(/[a-z\d\-\._~]*|(\?[a-z\d\-\._~]+=[a-z\d\-\._~]+))*",
      "NUM" : r"[+-]?((\d+(,|\.))*\d+)"}
  for k in regex.keys():
    txt = re.sub(regex[k], k, txt) 
  return txt

def tokenize(txt):
  tokens = nltk.word_tokenize(txt)
  prepostfix = lambda t: "<"+t+">" if t.isupper() else t
  return list(map(prepostfix, tokens))

In [93]:
df = df.dropna(subset =["type"])

df_content = df["content"]
df_cleaned = df_content.apply(clean_text)


df_token = df_cleaned.apply(tokenize)

df["tokens"] = df_token

#print(df_content.iloc[7])
#print(df_cleaned.iloc[7])
#print(df_token.iloc[7])

#print(df)


In [94]:
text = "  Bla sammen-sat 2019-22-01 Bla. bla, ?  +231 123 123.4 123,4 122.112,2 122,112.2  bla  gg@no.re   bla\nnew\t dr.dk dr.Einstein www.youtube.com youtube.com youtube.dk https://www.youtube.com/ \ttab  "


# Task 3 

In [207]:
def count_metatokens(tokens):
    meta_tokens  = ["<URL>", "<NUM>", "<DATE>", "<EMAIL>"]
    counts = [0, 0, 0, 0]
    for token in tokens:
      if token in meta_tokens:
        i = meta_tokens.index(token)
        counts[i] += 1
    return counts
            
def count_tokens(tokens):
    return len(tokens)

def count_all(tokens):
    cmt = count_metatokens(tokens)
    ct = count_tokens(tokens)
    return [ct] + cmt

def Observation1(df):
    tokencount = df['tokens'].apply(count_all)
    url_freqs = tokencount.apply(lambda xs: xs[1]/xs[0])
    url_freqs['type'] = df['type']
    types = url_freqs["type"].unique()
    freqs_types = []
    for t in types:
        url_freqs[url_freqs["type"] == t]
        #freqs_types.append((url_freqs[url_freqs["type"] == t]["tokens"]).mean(axis=1))
    return zip(types, freqs_types)
    
    

def Observation2(df):
    return df['type'].value_counts()
    
def Observation3(df):
    domains = df["domain"].unique()
    types = []
    for domain in domains:
        types.append(df[df["domain"] == domain]["type"].value_counts().to_dict())
    idx = sorted(range(len(types)), key=lambda i: list(types[i].items())[0][1])
    return list(map(lambda i: (domains[i], types[i]), reversed(idx)))
    
Observation1(df)

IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).